<a href="https://colab.research.google.com/github/alexwhite116/youtube-transcription/blob/main/youtube_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Transcription Project

This notebook looks to achieve the following:
1. Create/implement a model to transcribe a YouTube video given a link and store the transcription.
2. (Stretch) Implement a search function to enter text and return the most relevant video link based on its transcription.
3. (Stretch) Implement a summary of videos retrieved via the search function, also presenting the video title and thumbnail.
4. (Stretch) Implement automation for a specific streamer so that transcriptions/summaries of all of their content are available (and searchable).
5. Deploy the above as part of a Gradio app (stretch goals optional/iterative).